# 4.10 - Coding Etiquette & Excel Reporting Part 1.1

## This script contains the following points:

### 1. Importing data set

### 2. Data security check

### 3. Creating a regional segmentation of data

### 4. Creating an exclusion flag for high- and low-activity customers

# 1. Importing

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Create a path to main folder
path = r'C:\Users\nguye\OneDrive\02-2022 Instacart Basket Analysis'

In [3]:
# Import dataset
df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customers.pkl'))

In [4]:
# Verify datset
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,spender_flag,med_freq,frequency_flag,gender,state,age,number_of_dependants,family_status,income,exists
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,both


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 31 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   order_number            int64   
 3   orders_day_of_week      int64   
 4   order_hour_of_day       int64   
 5   days_since_prior_order  float64 
 6   product_id              int64   
 7   add_to_cart_order       int64   
 8   reordered               int64   
 9   product_name            object  
 10  aisle_id                int64   
 11  department_id           int64   
 12  prices                  float64 
 13  _merge                  category
 14  price_range_loc         object  
 15  busiest_day             object  
 16  traffic_status          object  
 17  busiest_period_of_day   object  
 18  max_order               int64   
 19  loyalty_flag            object  
 20  avg_price               float64 
 21  spende

In [6]:
df.shape

(32404859, 31)

# 2. Data Security

### The current dataset has no security implications. The "customers" data set had some PII, but these were removed prior to proceeding with the analysis to ensure customer privacy. 

# 3. Create a regional segmentation of the data

### Group the states into regions

In [7]:
# Check the state column
df['state'].value_counts().sort_index()

Alabama                 638003
Alaska                  648495
Arizona                 653964
Arkansas                636144
California              659783
Colorado                639280
Connecticut             623022
Delaware                637024
District of Columbia    613695
Florida                 629027
Georgia                 656389
Hawaii                  632901
Idaho                   607119
Illinois                633024
Indiana                 627282
Iowa                    625493
Kansas                  637538
Kentucky                632490
Louisiana               637482
Maine                   638583
Maryland                626579
Massachusetts           646358
Michigan                630928
Minnesota               647825
Mississippi             632675
Missouri                640732
Montana                 635265
Nebraska                625813
Nevada                  636139
New Hampshire           615378
New Jersey              627692
New Mexico              654494
New York

In [8]:
# Define the regions
northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
south = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
west = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [9]:
# Place the states into regions
df.loc[df['state'].isin(northeast), 'region'] = 'Northeast'
df.loc[df['state'].isin(midwest), 'region'] = 'Midwest'
df.loc[df['state'].isin(south), 'region'] = 'South'
df.loc[df['state'].isin(west), 'region'] = 'West'

In [10]:
# Check the region count
df['region'].value_counts(dropna = False)

South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: region, dtype: int64

In [11]:
# Check the length to verify all rows have been updated
len(df['region'])

32404859

### Check the spending habits across regions

In [12]:
# Create a crosstab between 'region' and 'spender_flag'
crosstab = pd.crosstab(df['region'], df['spender_flag'], dropna = False)

In [13]:
crosstab

spender_flag,High spender,Low spender
region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


# 4. Creating an exclusion flag for low-activity customers

In [14]:
# Create a flag for 'low-activity' customers
df.loc[df['max_order'] < 5, 'activity_flag'] = 'Low-activity'

In [15]:
# Create a flag for 'high-activity' customers
df.loc[df['max_order'] >= 5, 'activity_flag'] = 'High-activity'

In [16]:
df.tail(15)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,frequency_flag,gender,state,age,number_of_dependants,family_status,income,exists,region,activity_flag
32404844,1987169,106143,16,6,21,6.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,0,single,53755,both,West,High-activity
32404845,3143166,106143,17,0,8,15.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,0,single,53755,both,West,High-activity
32404846,3094081,106143,18,4,17,11.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,0,single,53755,both,West,High-activity
32404847,405982,106143,19,3,18,6.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,0,single,53755,both,West,High-activity
32404848,3102310,106143,20,3,16,7.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,0,single,53755,both,West,High-activity
32404849,1539810,106143,21,1,18,5.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,0,single,53755,both,West,High-activity
32404850,3308056,106143,22,4,20,10.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,0,single,53755,both,West,High-activity
32404851,2988973,106143,23,2,22,5.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,0,single,53755,both,West,High-activity
32404852,930,106143,24,6,12,4.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,0,single,53755,both,West,High-activity
32404853,467253,106143,25,6,16,7.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,0,single,53755,both,West,High-activity


In [17]:
# Verify that column is updated
df['activity_flag'].value_counts(dropna = False)

High-activity    30964564
Low-activity      1440295
Name: activity_flag, dtype: int64

In [18]:
len(df['activity_flag'])

32404859

In [19]:
# Create a new dataframe to store the high-activity customers
df_high_activity = df.loc[df['activity_flag'] == 'High-activity']

In [20]:
# Check dataframe
df_high_activity.head(20)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,frequency_flag,gender,state,age,number_of_dependants,family_status,income,exists,region,activity_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequent customer,Female,Alabama,31,3,married,40423,both,South,High-activity
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,3,married,40423,both,South,High-activity
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,3,married,40423,both,South,High-activity
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,3,married,40423,both,South,High-activity
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,3,married,40423,both,South,High-activity
5,3367565,1,6,2,7,19.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,3,married,40423,both,South,High-activity
6,550135,1,7,1,9,20.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,3,married,40423,both,South,High-activity
7,3108588,1,8,1,14,14.0,196,2,1,Soda,...,Non-frequent customer,Female,Alabama,31,3,married,40423,both,South,High-activity
8,2295261,1,9,1,16,0.0,196,4,1,Soda,...,Non-frequent customer,Female,Alabama,31,3,married,40423,both,South,High-activity
9,2550362,1,10,4,8,30.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,3,married,40423,both,South,High-activity


In [21]:
df_high_activity.shape

(30964564, 33)

In [22]:
# Export high activity dataframe
df_high_activity.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'high_activity_customers.pkl'))